In [0]:
CREATE OR REPLACE STREAMING TABLE bronze_food_inspection_dallas
  TBLPROPERTIES(
  'delta.enableChangeDataFeed' = 'true'
)
AS 
SELECT 
  * EXCEPT(street_direction, street_type, street_unit, street_number, street_name),
current_timestamp() as load_dt,
  _metadata.file_path as source_file_path,
  _metadata.file_name as source_file_name
FROM STREAM cloud_files('/Volumes/workspace/food_inspection_project/datastore',
  'csv',
    map(
    'cloudFiles.inferColumnTypes', 'true',
    'cloudFiles.schemaLocation', '/Volumes/workspace/food_inspection_project/datastore/bronze_food_inspection_dallas',
    'header', 'true'
  )
)


In [0]:
CREATE OR REFRESH STREAMING TABLE silver_food_inspection_dallas
AS
WITH base AS (
  SELECT
    -- core business columns
    restaurant_name,
    inspection_type,
    inspection_date,
    inspection_score,
    street_address,
    zip_code,
    inspection_month,
    inspection_year,
    fiscal_year,
    total_violation_score,
    latitude,
    longitude,

    -- keep ALL 25 violation fields as normal columns
    violation_description_1,  violation_points_1,  violation_detail_1,  violation_memo_1,
    violation_description_2,  violation_points_2,  violation_detail_2,  violation_memo_2,
    violation_description_3,  violation_points_3,  violation_detail_3,  violation_memo_3,
    violation_description_4,  violation_points_4,  violation_detail_4,  violation_memo_4,
    violation_description_5,  violation_points_5,  violation_detail_5,  violation_memo_5,
    violation_description_6,  violation_points_6,  violation_detail_6,  violation_memo_6,
    violation_description_7,  violation_points_7,  violation_detail_7,  violation_memo_7,
    violation_description_8,  violation_points_8,  violation_detail_8,  violation_memo_8,
    violation_description_9,  violation_points_9,  violation_detail_9,  violation_memo_9,
    violation_description_10, violation_points_10, violation_detail_10, violation_memo_10,
    violation_description_11, violation_points_11, violation_detail_11, violation_memo_11,
    violation_description_12, violation_points_12, violation_detail_12, violation_memo_12,
    violation_description_13, violation_points_13, violation_detail_13, violation_memo_13,
    violation_description_14, violation_points_14, violation_detail_14, violation_memo_14,
    violation_description_15, violation_points_15, violation_detail_15, violation_memo_15,
    violation_description_16, violation_points_16, violation_detail_16, violation_memo_16,
    violation_description_17, violation_points_17, violation_detail_17, violation_memo_17,
    violation_description_18, violation_points_18, violation_detail_18, violation_memo_18,
    violation_description_19, violation_points_19, violation_detail_19, violation_memo_19,
    violation_description_20, violation_points_20, violation_detail_20, violation_memo_20,
    violation_description_21, violation_points_21, violation_detail_21, violation_memo_21,
    violation_description_22, violation_points_22, violation_detail_22, violation_memo_22,
    violation_description_23, violation_points_23, violation_detail_23, violation_memo_23,
    violation_description_24, violation_points_24, violation_detail_24, violation_memo_24,
    violation_description_25, violation_points_25, violation_detail_25, violation_memo_25,

    -- arrays for stats
    array(
      violation_description_1,  violation_description_2,  violation_description_3,
      violation_description_4,  violation_description_5,  violation_description_6,
      violation_description_7,  violation_description_8,  violation_description_9,
      violation_description_10, violation_description_11, violation_description_12,
      violation_description_13, violation_description_14, violation_description_15,
      violation_description_16, violation_description_17, violation_description_18,
      violation_description_19, violation_description_20, violation_description_21,
      violation_description_22, violation_description_23, violation_description_24,
      violation_description_25
    ) AS violation_desc_array,

    array(
      violation_points_1,  violation_points_2,  violation_points_3,
      violation_points_4,  violation_points_5,  violation_points_6,
      violation_points_7,  violation_points_8,  violation_points_9,
      violation_points_10, violation_points_11, violation_points_12,
      violation_points_13, violation_points_14, violation_points_15,
      violation_points_16, violation_points_17, violation_points_18,
      violation_points_19, violation_points_20, violation_points_21,
      violation_points_22, violation_points_23, violation_points_24,
      violation_points_25
    ) AS violation_points_array,

    array(
      violation_detail_1,  violation_detail_2,  violation_detail_3,
      violation_detail_4,  violation_detail_5,  violation_detail_6,
      violation_detail_7,  violation_detail_8,  violation_detail_9,
      violation_detail_10, violation_detail_11, violation_detail_12,
      violation_detail_13, violation_detail_14, violation_detail_15,
      violation_detail_16, violation_detail_17, violation_detail_18,
      violation_detail_19, violation_detail_20, violation_detail_21,
      violation_detail_22, violation_detail_23, violation_detail_24,
      violation_detail_25
    ) AS violation_detail_array,

    array(
      violation_memo_1,  violation_memo_2,  violation_memo_3,
      violation_memo_4,  violation_memo_5,  violation_memo_6,
      violation_memo_7,  violation_memo_8,  violation_memo_9,
      violation_memo_10, violation_memo_11, violation_memo_12,
      violation_memo_13, violation_memo_14, violation_memo_15,
      violation_memo_16, violation_memo_17, violation_memo_18,
      violation_memo_19, violation_memo_20, violation_memo_21,
      violation_memo_22, violation_memo_23, violation_memo_24,
      violation_memo_25
    ) AS violation_memo_array,

    load_dt,
    source_file_path,
    source_file_name
  FROM STREAM bronze_food_inspection_dallas
),

viol_stats AS (
  SELECT
    *,
    -- R3: count distinct non-null violations
    size(
      array_distinct(
        filter(violation_desc_array, v -> v IS NOT NULL)
      )
    ) AS violation_count
  FROM base
)

SELECT
  restaurant_name,
  inspection_type,
  inspection_date,
  inspection_score,
  street_address,
  zip_code,
  inspection_month,
  inspection_year,
  fiscal_year,
  total_violation_score,
  violation_count,
  latitude,
  longitude,
  load_dt,
  source_file_path,
  source_file_name,

  -- all 25 violation fields
  violation_description_1,  violation_points_1,  violation_detail_1,  violation_memo_1,
  violation_description_2,  violation_points_2,  violation_detail_2,  violation_memo_2,
  violation_description_3,  violation_points_3,  violation_detail_3,  violation_memo_3,
  violation_description_4,  violation_points_4,  violation_detail_4,  violation_memo_4,
  violation_description_5,  violation_points_5,  violation_detail_5,  violation_memo_5,
  violation_description_6,  violation_points_6,  violation_detail_6,  violation_memo_6,
  violation_description_7,  violation_points_7,  violation_detail_7,  violation_memo_7,
  violation_description_8,  violation_points_8,  violation_detail_8,  violation_memo_8,
  violation_description_9,  violation_points_9,  violation_detail_9,  violation_memo_9,
  violation_description_10, violation_points_10, violation_detail_10, violation_memo_10,
  violation_description_11, violation_points_11, violation_detail_11, violation_memo_11,
  violation_description_12, violation_points_12, violation_detail_12, violation_memo_12,
  violation_description_13, violation_points_13, violation_detail_13, violation_memo_13,
  violation_description_14, violation_points_14, violation_detail_14, violation_memo_14,
  violation_description_15, violation_points_15, violation_detail_15, violation_memo_15,
  violation_description_16, violation_points_16, violation_detail_16, violation_memo_16,
  violation_description_17, violation_points_17, violation_detail_17, violation_memo_17,
  violation_description_18, violation_points_18, violation_detail_18, violation_memo_18,
  violation_description_19, violation_points_19, violation_detail_19, violation_memo_19,
  violation_description_20, violation_points_20, violation_detail_20, violation_memo_20,
  violation_description_21, violation_points_21, violation_detail_21, violation_memo_21,
  violation_description_22, violation_points_22, violation_detail_22, violation_memo_22,
  violation_description_23, violation_points_23, violation_detail_23, violation_memo_23,
  violation_description_24, violation_points_24, violation_detail_24, violation_memo_24,
  violation_description_25, violation_points_25, violation_detail_25, violation_memo_25

FROM viol_stats
WHERE
  -- R1
  restaurant_name   IS NOT NULL
  AND inspection_date  IS NOT NULL
  AND inspection_type  IS NOT NULL
  AND zip_code         IS NOT NULL

  -- R2
  AND (total_violation_score <= 100 OR total_violation_score IS NULL)

  -- R3
  AND violation_count >= 1

  -- R4
  AND NOT (total_violation_score >= 90 AND violation_count > 3);

In [0]:
CREATE OR REFRESH LIVE TABLE silver_business
AS
WITH distinct_business AS (
  SELECT DISTINCT
    restaurant_name
  FROM silver_food_inspection_dallas       
  WHERE restaurant_name IS NOT NULL
)
SELECT
  DENSE_RANK() OVER (ORDER BY restaurant_name) AS business_key,   
  restaurant_name
FROM distinct_business;

In [0]:
CREATE OR REFRESH LIVE TABLE silver_inspection_type
AS
WITH distinct_types AS (
  SELECT DISTINCT
    inspection_type
  FROM silver_food_inspection_dallas            
  WHERE inspection_type IS NOT NULL
)
SELECT
  DENSE_RANK() OVER (ORDER BY inspection_type) AS inspection_type_key,   
  inspection_type
FROM distinct_types;

In [0]:
CREATE OR REFRESH LIVE TABLE silver_violation
AS
WITH base AS (
  SELECT
    array(
      violation_description_1, violation_description_2, violation_description_3,
      violation_description_4, violation_description_5, violation_description_6,
      violation_description_7, violation_description_8, violation_description_9,
      violation_description_10, violation_description_11, violation_description_12,
      violation_description_13, violation_description_14, violation_description_15,
      violation_description_16, violation_description_17, violation_description_18,
      violation_description_19, violation_description_20, violation_description_21,
      violation_description_22, violation_description_23, violation_description_24,
      violation_description_25
    ) AS desc_array,
    array(
      violation_points_1, violation_points_2, violation_points_3,
      violation_points_4, violation_points_5, violation_points_6,
      violation_points_7, violation_points_8, violation_points_9,
      violation_points_10, violation_points_11, violation_points_12,
      violation_points_13, violation_points_14, violation_points_15,
      violation_points_16, violation_points_17, violation_points_18,
      violation_points_19, violation_points_20, violation_points_21,
      violation_points_22, violation_points_23, violation_points_24,
      violation_points_25
    ) AS points_array,
    array(
      violation_detail_1, violation_detail_2, violation_detail_3,
      violation_detail_4, violation_detail_5, violation_detail_6,
      violation_detail_7, violation_detail_8, violation_detail_9,
      violation_detail_10, violation_detail_11, violation_detail_12,
      violation_detail_13, violation_detail_14, violation_detail_15,
      violation_detail_16, violation_detail_17, violation_detail_18,
      violation_detail_19, violation_detail_20, violation_detail_21,
      violation_detail_22, violation_detail_23, violation_detail_24,
      violation_detail_25
    ) AS detail_array,
    array(
      violation_memo_1, violation_memo_2, violation_memo_3,
      violation_memo_4, violation_memo_5, violation_memo_6,
      violation_memo_7, violation_memo_8, violation_memo_9,
      violation_memo_10, violation_memo_11, violation_memo_12,
      violation_memo_13, violation_memo_14, violation_memo_15,
      violation_memo_16, violation_memo_17, violation_memo_18,
      violation_memo_19, violation_memo_20, violation_memo_21,
      violation_memo_22, violation_memo_23, violation_memo_24,
      violation_memo_25
    ) AS memo_array
  FROM silver_food_inspection_dallas
),
exploded AS (
  SELECT
    posexplode_outer(desc_array) AS (idx, violation_description),
    points_array,
    detail_array,
    memo_array
  FROM base
),
flat AS (
  SELECT
    violation_description,
    points_array[idx]  AS violation_points,
    detail_array[idx]  AS violation_detail,
    memo_array[idx]    AS violation_memo
  FROM exploded
  WHERE violation_description IS NOT NULL              
)
SELECT DISTINCT                                          
  -- handles "*24 ..." or "24 ..." → captures 24
  regexp_extract(violation_description, '^\\*?(\\d+)', 1) AS violation_code,
  violation_description,
  violation_points,
  violation_detail,
  violation_memo
FROM flat
WHERE regexp_extract(violation_description, '^\\*?(\\d+)', 1) <> '';  

In [0]:
CREATE OR REFRESH LIVE TABLE silver_date
AS
WITH days AS (
  SELECT explode(sequence(date_sub(current_date(), 5*365), current_date(), interval 1 day)) AS d
)
SELECT
  CAST(date_format(d, 'yyyyMMdd') AS INT) AS date_key,
  CAST(d AS DATE)                       AS full_date,
  year(d)                                      AS year,
  quarter(d)                                   AS quarter,
  date_format(d, 'MMMM')                       AS month_name,  
  day(d)                                       AS day,
  dayofweek(d)                                 AS day_of_week
FROM days;

In [0]:
CREATE OR REPLACE LIVE TABLE silver_geo
AS
SELECT
  try_cast(split(cast(zipcode AS STRING), '\\.')[0] AS INT) AS zip_code,
  city,
  state
FROM read_files(
  "/Volumes/workspace/spl/datastore/geo/geo-data.csv",
  format => "csv",
  header => "true",
  inferSchema => "true"
);